In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
import time



C:\Users\siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Step 1: Load and preprocess the dataset
df = pd.read_csv('IMDB Dataset.csv')


In [3]:

# Map 'positive' to 1 and 'negative' to 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['review'])
X = tokenizer.texts_to_sequences(df['review'])
X = pad_sequences(X, padding='pre', maxlen=100)
y = df['sentiment']



In [4]:
# Step 2: Define the deep neural network model architecture
embedding_dim = 128
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=embedding_dim, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))



C:\Users\siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [5]:
# Step 3: Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time:.2f} seconds")



Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 71s 109ms/step - accuracy: 0.7362 - loss: 0.5142 - val_accuracy: 0.8509 - val_loss: 0.3599
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 90s 143ms/step - accuracy: 0.8595 - loss: 0.3366 - val_accuracy: 0.8604 - val_loss: 0.3277
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 152s 159ms/step - accuracy: 0.8783 - loss: 0.2981 - val_accuracy: 0.8644 - val_loss: 0.3149
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 101s 161ms/step - accuracy: 0.8943 - loss: 0.2640 - val_accuracy: 0.8642 - val_loss: 0.3191
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 66s 105ms/step - accuracy: 0.9061 - loss: 0.2338 - val_accuracy: 0.8668 - val_loss: 0.3311
Training time: 478.83 seconds


In [6]:
# Step 5: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')



313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.8635 - loss: 0.3351
Test Loss: 0.3311004638671875
Test Accuracy: 0.8668000102043152


In [8]:
# Step 6: Make predictions on new data (dynamic input)
while True:
    user_input = input("Enter a movie review: ")
    # Preprocess the user input
    input_sequence = tokenizer.texts_to_sequences([user_input])
    input_sequence = pad_sequences(input_sequence, padding='pre', maxlen=100)
    
    start_time = time.time()
    # Make prediction
    prediction = model.predict(input_sequence)[0][0]
    end_time = time.time()
    
    # Print prediction
    if prediction >= 0.5:
        print("Predicted sentiment: Positive")
    else:
        print("Predicted sentiment: Negative")
    
    prediction_time = end_time - start_time
    print(f"Prediction time: {prediction_time:.2f} seconds")


Enter a movie review:  In my mind, this is clearly the best short film out of the three that aired under the same programme. The story has an amazing build up and the ending is absolutely unpredictable!! That's why I loved this one the most. You are left in awe at the end. The other two short films had pretty predictable endings. And have to admire the amazing acting by Tina Desai and Kunal with amazing direction by Sujoy. Overall a 10/10 from me.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Predicted sentiment: Positive
Prediction time: 0.29 seconds


Enter a movie review:  This is the best example of a movie with a great premise and a useless unsatisfactory ending. The build up of the story was great and a lot of suspense was created but towards the end the story had many loopholes and the ending was incomplete. My mom says the ending was like a black hole - you are nowhere after reaching the ending. The director left us clueless. Khoda pahaad nikla Chopra


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted sentiment: Positive
Prediction time: 0.08 seconds


Enter a movie review:  I'm not sure what accomplished director/producer/cinematographer Joshua Caldwell was thinking taking on this project.  This film has got to be the epitome of terrible writing and should be a classroom example of 'what not to do' when writing a screenplay. Why would Joshua take on (clearly) amateur writer Adam Gaines script is beyond me. Even his good directing and excellent cinematography could not save this disaster.  Aside from the super obvious plot holes and very poor story overall, the dragged-out unnecessary dialogue made this film unbearable and extremely boring. The way too long 1h 39min film length felt like 4 hours and I found myself saying "get on with it already, who cares!" when the two leads would just ramble on about nothing relevant. This movie may have been interesting if it was a 30 min short film (which oddly enough is the only minimal writing experience Adam Gaines has).  The acting was decent and Katia Winter was very easy on the eyes to look

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted sentiment: Negative
Prediction time: 0.08 seconds


KeyboardInterrupt: Interrupted by user